### Introduction
There are two (fictitious) independent store chains selling Kaggle merchandise that want to become the official outlet for all things Kaggle. The goal of the project is to help Kaggle figure out which of the store chains would have the best sales going forward. So, Kaggle has collected some data and is asking us to build forecasting models to help us decide.

Help Kaggle figure out whether KaggleMart or KaggleRama should become the official Kaggle outlet!

Kaggle competition link: https://www.kaggle.com/c/tabular-playground-series-jan-2022/overview

### Download dataset from Kaggle if RELOAD flag is True

In [1]:
# Flag to force to reload dataset
RELOAD = False

In [2]:
# import Kaggle API to load dataset
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

In [3]:
# initialize Kaggle API
api = KaggleApi()
api.authenticate()

In [4]:
# download dataset from Kaggle to data folder
data_path = 'data'
api.competition_download_files('tabular-playground-series-jan-2022', data_path, force=RELOAD)
# save filename: !ATTENTION! : it may not be wroking if many files are in folders
# then just name it manually 
dataset_file_name = os.listdir(data_path)[0]

In [5]:
# import libraries to work with paths and to read zipped file, as was downloaded from Kaggle
import os.path
from zipfile import ZipFile
# import pandas for EDA
import pandas as pd
# Read train dataset (train.csv) to pandas DataFrame named df: it will be used for analysis
df = pd.read_csv(ZipFile(os.path.join(data_path, dataset_file_name)).open('train.csv'))
# Read test dataset (test.csv), to pandas DataFrame named df_validation. It will be used only to generate final predictions, which will be submitted
df_validation = pd.read_csv(ZipFile(os.path.join(data_path, dataset_file_name)).open('test.csv'))
# finally, we will download example of submission (there are no correct predictions there, it is just an example)
df_subm_example = pd.read_csv(ZipFile(os.path.join(data_path, dataset_file_name)).open('sample_submission.csv'))

# Check, that all dataframes are loaded and have correct shapes (train should have six columns, 
# test should have 5 columns - no outcome, and sample submission should have two columns)
print('Shape of df: ' + str(df.shape))
print('Shape of df_validation: ' + str(df_validation.shape))
print('Shape of df_subm_example: ' + str(df_subm_example.shape))

Shape of df: (26298, 6)
Shape of df_validation: (6570, 5)
Shape of df_subm_example: (6570, 2)


### Exploratory Data Analysis

In [6]:
# first look on dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26298 entries, 0 to 26297
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   row_id    26298 non-null  int64 
 1   date      26298 non-null  object
 2   country   26298 non-null  object
 3   store     26298 non-null  object
 4   product   26298 non-null  object
 5   num_sold  26298 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


Train dataset contains 26298 entries and 6 columns. Let's take a look on each of them.

In [7]:
# row_id
print('row_id type: ' + str(df.row_id.dtype))
# it is an integer value

# head, to check how data looks like
print('row_id head: \n' + str(df.row_id.head()))
# looks like just an oreder number, let's check tail
print('row_id tail: \n' + str(df.row_id.tail()))

row_id type: int64
row_id head: 
0    0
1    1
2    2
3    3
4    4
Name: row_id, dtype: int64
row_id tail: 
26293    26293
26294    26294
26295    26295
26296    26296
26297    26297
Name: row_id, dtype: int64


In [8]:
# check number of unique values
df.row_id.nunique() == len(df)

#df_row contains id of data entry

True

In [9]:
# Check for NaN:
df.row_id.isna().sum()

0

In [10]:
# date column
# type is object
# let's check head and tail:
print('date head: \n' + str(df.date.head()))
print('date tail: \n' + str(df.date.tail()))

date head: 
0    2015-01-01
1    2015-01-01
2    2015-01-01
3    2015-01-01
4    2015-01-01
Name: date, dtype: object
date tail: 
26293    2018-12-31
26294    2018-12-31
26295    2018-12-31
26296    2018-12-31
26297    2018-12-31
Name: date, dtype: object


In [11]:
# date column has dates of entries from 1.1.2015 till 31.12.2018
# number of unique dates:
df.date.nunique()

1461

In [12]:
# Check for NaN:
df.date.isna().sum()

0

In [13]:
# check type of single entry in the column
type(df.date[0])

str

In [14]:
# how many records on each date?
print('Minimum values of records in one date: \n' + str(df.groupby('date').count().max()))
print('Maximum values of records in one date: \n' + str(df.groupby('date').count().min()))

# Each day contains 18 records

Minimum values of records in one date: 
row_id      18
country     18
store       18
product     18
num_sold    18
dtype: int64
Maximum values of records in one date: 
row_id      18
country     18
store       18
product     18
num_sold    18
dtype: int64


In [15]:
# Check Country column
# type is object
# let's check head and tail:
print('country head: \n' + str(df.country.head()))
print('country tail: \n' + str(df.country.tail()))

country head: 
0    Finland
1    Finland
2    Finland
3    Finland
4    Finland
Name: country, dtype: object
country tail: 
26293    Sweden
26294    Sweden
26295    Sweden
26296    Sweden
26297    Sweden
Name: country, dtype: object


In [16]:
# number of unique countries:
df.country.nunique()

3

In [17]:
# only three unique values, let's print them:
df.country.unique()

array(['Finland', 'Norway', 'Sweden'], dtype=object)

In [18]:
# Check for Nan
df.country.isna().sum()

0

In [19]:
# Country column represents location of the store. Let's check how many entries from each country we have
df.groupby('country').count()

# For each country it is equal amount of records (8766)

,row_id,date,store,product,num_sold
country,,,,,
Finland,8766,8766,8766,8766,8766
Norway,8766,8766,8766,8766,8766
Sweden,8766,8766,8766,8766,8766


In [20]:
# store column
# type is object
# let's check head and tail:
print('store head: \n' + str(df.store.head()))
print('store tail: \n' + str(df.store.tail()))

store head: 
0    KaggleMart
1    KaggleMart
2    KaggleMart
3    KaggleRama
4    KaggleRama
Name: store, dtype: object
store tail: 
26293    KaggleMart
26294    KaggleMart
26295    KaggleRama
26296    KaggleRama
26297    KaggleRama
Name: store, dtype: object


In [21]:
# number of unique stores:
df.store.nunique()

2

In [22]:
# only two unique values, let's print them:
df.store.unique()

array(['KaggleMart', 'KaggleRama'], dtype=object)

In [23]:
# Check for Nan
df.store.isna().sum()

0

In [24]:
# Let's check how many entries from each store we have
df.groupby('store').count()

# For each store it is equal amount of records (13149)

,row_id,date,country,product,num_sold
store,,,,,
KaggleMart,13149,13149,13149,13149,13149
KaggleRama,13149,13149,13149,13149,13149


In [25]:
# product column
# type is object
# let's check head and tail:
print('product head: \n' + str(df['product'].head()))
print('product tail: \n' + str(df['product'].tail()))

product head: 
0        Kaggle Mug
1        Kaggle Hat
2    Kaggle Sticker
3        Kaggle Mug
4        Kaggle Hat
Name: product, dtype: object
product tail: 
26293        Kaggle Hat
26294    Kaggle Sticker
26295        Kaggle Mug
26296        Kaggle Hat
26297    Kaggle Sticker
Name: product, dtype: object


In [26]:
# number of unique products:
df['product'].nunique()

3

In [27]:
# only three unique values, let's print them:
df['product'].unique()

array(['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker'], dtype=object)

In [28]:
# Check for Nan
df['product'].isna().sum()

0

In [29]:
# Let's check how many entries from each product we have
df.groupby('product').count()

# For each product it is equal amount of records (8766)

,row_id,date,country,store,num_sold
product,,,,,
Kaggle Hat,8766,8766,8766,8766,8766
Kaggle Mug,8766,8766,8766,8766,8766
Kaggle Sticker,8766,8766,8766,8766,8766


In [30]:
# num_sold column
print('num_sold type: ' + str(df.num_sold.dtype))
# it is an integer value
# let's check head and tail:
print('num_sold head: \n' + str(df.num_sold.head()))
print('num_sold tail: \n' + str(df.num_sold.tail()))

num_sold type: int64
num_sold head: 
0    329
1    520
2    146
3    572
4    911
Name: num_sold, dtype: int64
num_sold tail: 
26293     823
26294     250
26295    1004
26296    1441
26297     388
Name: num_sold, dtype: int64


In [31]:
# number of unique stores:
df.num_sold.nunique()

1377

In [32]:
# Check for Nan
df.num_sold.isna().sum()

0